# 藝術風格化模型應用

In [ ]:
!gdown '1hsPe_rD4YXspLP2Ms4-iYe3gw4jqLNN2' --output style-transfer.zip
!unzip style-transfer.zip

## 設定環境
TensorFlow Hub 是已訓練機器學習模型的存放區

In [ ]:
#載入需要用到的套件
import os
import functools
from matplotlib import gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub

In [ ]:
print("TF 版本: ", tf.__version__)
print("TF-Hub 版本: ", hub.__version__)
print("Eager mode 是否啟用: ", tf.executing_eagerly())
print("GPU 可用情形: ", tf.config.list_physical_devices('GPU'))

## upload local image

In [ ]:
# prompt: import image from local

from google.colab import files
uploaded = files.upload()


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 影像前處理函式

In [ ]:
# @title 定義圖像加載和可視化功能
#Define image loading and visualization functions  { display-mode: "form" }
# 剪裁圖片
def crop_center(image):
  """回傳一個已裁剪好的正方形圖片."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

# 載入圖片
@functools.lru_cache(maxsize=None) #高阶函数,如果maxsize设置为NoneLRU功能将被禁用且缓存数量无上限。
def load_image(image_path, image_size=(256, 256), preserve_aspect_ratio=True):
  img = plt.imread(image_path).astype(np.float32)[np.newaxis, ...]
  if img.max() > 1.0:
    img = img / 255.
  if len(img.shape) == 3:
    img = tf.stack([img, img, img], axis=-1)
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

# 顯示圖片
def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w  * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()

In [ ]:
# @title 加載範例圖片 { display-mode: "form" }
content_image = 'myphoto.jpg' # @param {type:"string"}
style_image = 'style2.jpg' # @param {type:"string"}
output_image_size = 384  # @param {type:"integer"}

content_img_size = (output_image_size, output_image_size)
style_img_size = (256, 256)
content_image = load_image(content_image, content_img_size)
style_image = load_image(style_image, style_img_size)
style_image = tf.nn.avg_pool(style_image, ksize=[3,3], strides=[1,1], padding='SAME')
show_n([content_image, style_image], ['Content image', 'Style image'])

## 導入TF-Hub模塊(需要等待一點時間)
https://www.tensorflow.org/hub?hl=zh-tw

https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2

In [ ]:
# 導入TF-Hub模塊

hub_handle = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
hub_module = hub.load(hub_handle)

## 展示一張的圖像風格(需要約8秒)

In [ ]:
outputs = hub_module(content_image, style_image)
stylized_image = outputs[0]

show_n([content_image, style_image, stylized_image],
    titles=['Original content image', 'Style image', 'Stylized image'])

## 練習轉一張圖
1. 上傳圖檔後

2. **到 [加載示例圖片] 輸入圖檔名稱**

3. 展示一張的圖像風格

動漫藝術家作品網站:
https://www.artstation.com/?sort_by=trending

風格移轉的 tensorflow hub 教學網站
https://www.tensorflow.org/hub/tutorials/tf2_arbitrary_image_stylization?hl=zh-tw

In [ ]:
# @title caculate traingle total
length = '52' # @param {type:'string'} ["53",'52']
hight = 23  # @param {type:'integer'}
area = (int(length) * hight) / 2
print(f'traingle hight = {area}')


## 定義圖像網址加載和可視化功能

In [ ]:
# @title 定義圖像網址加載和可視化功能
#Define image loading and visualization functions  { display-mode: "form" }
# 剪裁圖片
def crop_center(image):
  """回傳一個已裁剪好的正方形圖片."""
  shape = image.shape
  new_shape = min(shape[1], shape[2])
  offset_y = max(shape[1] - shape[2], 0) // 2
  offset_x = max(shape[2] - shape[1], 0) // 2
  image = tf.image.crop_to_bounding_box(
      image, offset_y, offset_x, new_shape, new_shape)
  return image

# 載入圖片
@functools.lru_cache(maxsize=None) #高阶函数,如果maxsize设置为NoneLRU功能将被禁用且缓存数量无上限。
def load_image(image_url, image_size=(256, 256), preserve_aspect_ratio=True):
  image_path = tf.keras.utils.get_file(os.path.basename(image_url)[-128:], image_url)
  img = tf.io.decode_image(tf.io.read_file(image_path), channels = 3, dtype = tf.float32)[np.newaxis, ...]
  img = crop_center(img)
  img = tf.image.resize(img, image_size, preserve_aspect_ratio=True)
  return img

# 顯示圖片
def show_n(images, titles=('',)):
  n = len(images)
  image_sizes = [image.shape[1] for image in images]
  w = (image_sizes[0] * 6) // 320
  plt.figure(figsize=(w  * n, w))
  gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
  for i in range(n):
    plt.subplot(gs[i])
    plt.imshow(images[i][0], aspect='equal')
    plt.axis('off')
    plt.title(titles[i] if len(titles) > i else '')
  plt.show()